<a href="https://colab.research.google.com/github/rubaahmedkhan/AgenticAI-Toolkit-/blob/main/agent_as_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  -qU openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.6/734.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata


In [ ]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-1.5-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [ ]:
from agents.tool import function_tool

In [ ]:
@function_tool("get_weather")
def get_weather(location: str, unit: str = "C") -> str:
  """
  You are a Web Development Assistant. You only respond to queries related to web development, including front-end and back-end technologies,
  frameworks (like HTML, CSS, JavaScript, React, Node.js, etc.), APIs, databases, hosting, and deployment. You must not respond to questions
   outside the scope of web development. For unrelated queries, politely inform the user that your role is limited to web development assistance only.
  """
  # Example logic
  return f"The weather in {location} is 22 degrees {unit}."


In [ ]:
@function_tool("piaic_student_finder")
def student_finder(student_roll: int) -> str:
  """
  find the PIAIC student based on the roll number
  """
  data = {1: "Qasim",
          2: "Sir Zia",
          3: "Daniyal"}

  return data.get(student_roll, "Not Found")

In [ ]:
@function_tool("get_weather")
def get_weather(location: str, unit: str = "C") -> str:
  """
  Fetch the weather for a given location, returning a short description.
  """
  # Example logic
  return f"The weather in {location} is 22 degrees {unit}."


In [ ]:
import asyncio

from agents import Agent, Runner

web_development_expert:Agent = Agent(
        name="Web Development Agent",
        instructions="You are a Web Development Assistant. You only respond to queries related to web development,",
        model=model,
        handoff_description="Web development expert"
    )

mobile_development_expert:Agent = Agent(
        name="mobile Development Agent",
        instructions="You are a mobile Development Assistant. You only respond to queries related to mobile development,",
        model=model,
        handoff_description="mobile development expert"
    )
Devops_expert:Agent = Agent(
        name="Devops Agent",
        instructions="You are a Devops Assistant. You only respond to queries related to Devops,",
        model=model,
        handoff_description="Devops expert"
    )
Openai_agent:Agent = Agent(
        name="OpenAI Agent",
        instructions="You are a OpenAI Assistant. You only respond to queries related to OpenAI,",
        model=model,
        handoff_description="OpenAI expert"
    )
Agentic_AI_expert:Agent = Agent(
        name="AI Development Agent",
        instructions="""You are an AI Development Assistant. You only respond to queries related to AI development. You have access to two tools to assist with your tasks Agentic_AI –
                     for handling questions related to autonomous or agent-based AI systems

                     DevOps: for managing AI deployment, CI/CD, and infrastructure-related tasks


                     Agentic_AI: for handling questions related to autonomous or agent-based AI systems
                     Use these tools when appropriate to provide accurate and efficient support
                    """,
        tools=[
        Devops_expert.as_tool(
            tool_name="Devops",
            tool_description="provide detail answer related to agentic ai related queries",
        ),
        Openai_agent.as_tool(
            tool_name="open AI expert",
            tool_description="provide detail answer related to agentic ai related queries",
        ),

    ],
        model=model,
        handoff_description="AI development expert"
    )

triage_agent:Agent = Agent(
        name="Triage agent",
        instructions="Redirect all questions related to web development, mobile development, or Agentic AI to their respective agents: web_development_expert, mobile_development_expert, and Agentic_AI_expert..",
        model=model,
        handoffs=[web_development_expert,mobile_development_expert,Agentic_AI_expert]
    )

result = await Runner.run(triage_agent, "describe devops.")
print(result.final_output)
print(result.last_agent)




DevOps is a set of practices, tools, and a cultural philosophy that automates and integrates the processes between software development and IT teams.  It aims to shorten the systems development life cycle and provide continuous delivery with high software quality.  This is achieved through a collaborative approach that emphasizes communication, shared responsibility, and automation.  Key aspects include continuous integration, continuous delivery/continuous deployment (CI/CD), infrastructure as code, and monitoring.

Agent(name='Triage agent', instructions='Redirect all questions related to web development, mobile development, or Agentic AI to their respective agents: web_development_expert, mobile_development_expert, and Agentic_AI_expert..', prompt=None, handoff_description=None, handoffs=[Agent(name='Web Development Agent', instructions='You are a Web Development Assistant. You only respond to queries related to web development,', prompt=None, handoff_description='Web development ex